#Chapter2

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
%cd /content/drive/MyDrive/bioinf/podaci/

/content/drive/MyDrive/bioinf/podaci


In [40]:
import numpy as np
import random
from random import randint

#    

## [BA2A](https://rosalind.info/problems/ba2a/) - Implement MotifEnumeration

In [3]:
def neighbour(pattern, mismatch, words): # BA1I
  if mismatch == 0: # ako ako je preostali broj mogucih razlicitih 0
    # dodaj sam uzorak
    words.add(pattern)
  # popis baza
  bases = ['A', 'C', 'G', 'T']
  for i in range(len(pattern)): # iteriraj po dobivenom uzorku
    for j in range(len(bases)): # iteriraj po bazama
      # novi uzorak je onaj s promjenutom bazom na nekoj od lokacija
      new_pattern = pattern[:i] + bases[j] + pattern[i+1:]
      if mismatch <= 1: # ako je preostali broj mogucih razlicitih manji od 1
        words.add(new_pattern) # dodaj novi dobiveni uzorak
        # inace
      else:
        # rekurzivno pozovi pretragu al sa smanjenim brojem dopustenih razlika
        neighbour(new_pattern, mismatch - 1, words)

In [6]:
def motifEnumeration(dna, k, d):
  patterns = []
  for n in range(len(dna)): # za svaki dna u redu
    pattern = set() # postavi prazan skup
    for i in range(len(dna[n]) - k + 1): # iteriraj po jednom dna
      kmerspattern = set() # prazan skup za spremanje "susjeda"
      # pronadi susjede s navjise d razlika od svakog substringa
      neighbour(dna[n][i:i + k], d, kmerspattern)
      # za svaku dobivenu "rijec" - kmer iz dobivenog skupa
      for words in kmerspattern:
        # dodaj "rijeci" - kmere u skup
        pattern.add(words)
    for j in pattern: # za svaki k-mer u dobivenom skupu
      patterns.append(j) # dodaj u niz s pocetka
  motifpattern = [] # lista za konacni rezultat
  for element in patterns: # za svaki k-mer u listi
    # ako je broj pojavljivanja k-mera jednak broju elemenata u listi DNA
    if patterns.count(element) == len(dna):
      # dodaj element
      motifpattern.append(element)
  # stavi u listu skup jer on izbacuje duplikate
  motifpattern = list(set(motifpattern))
  motifpattern.sort()
  return ' '.join(motifpattern)

dna = ['ATTTGGC', 'TGCCTTA', 'CGGTATC', 'GAAAATT']
motifEnumeration(dna, 3,1)

'ATA ATT GTT TTT'

In [9]:
with open("rosalind_ba2a.txt", "r") as f:
  k, d = map(int, f.readline().strip().split())
  dna = [line.strip() for line in f]
motifEnumeration(dna, k, d)

'GACAA GACCA GACGA GACTA'

## [BA2B](https://rosalind.info/problems/ba2b/) - Find a Median String

In [10]:
# Hamming Distance - BA1G
def minHammingDistance(pattern, string):
  # postavi minimalnu udaljenost na cijelu duljinu uzorka
  min_distance = len(pattern)
  for i in range(len(string) - len(pattern) + 1): # iteriranje po stringu
    # izracunaj Hamming udaljenost
    # zbroji sve jedinice koje dobivamo pri usporedbi substringa  i patterna
    substring = string[i: i + len(pattern)]
    distance = sum([1 for j in range(len(pattern))
                    if pattern[j] != substring[j]])
    # ako je Hamming udaljenost manja od minimalne udaljenosti
    # (pocetna je duljina uzorka)
    if distance < min_distance:
      min_distance = distance # postavi minimalnu udaljenost na novo izracunatu
  return min_distance # vrati minimalnu Hamming udaljenost

In [11]:
def generateArray(k): # sve perm duljine k
  bases = ['A', 'C', 'G', 'T'] # lista bazi
  array = bases.copy() # lista bazi
  for n in range(k-1): # za svaki n do k-1
    # spoji i-ti i j-ti element iz array i bases i dodaj u niz
    # bases ce uvijek dodati samo jedan element na postojece
    array = [i+j for i in array for j in bases]
  return array

In [16]:
# dvostruki min
def findMedianString(k, dna):
  patterns = generateArray(k)
  distances = {}
  min_distance = len(dna) * len(patterns) # najveca moguca udaljenost
  output=[]
  for pattern in patterns: # za svaki uzorak
    sum_distance = 0 # ukupna udaljenost je 0
    for j in range(len(dna)): # za svaki DNA u nizu
      # zbroji minimalne Hamming udaljenosti izmedu uzorka iz perm i DNA u nizu
      sum_distance += minHammingDistance(pattern, dna[j])
    # spremi min udaljenost za odredenu perm
    distances[pattern] = sum_distance
    if sum_distance < min_distance: # ako je dobivena najmanja Hamming udaljenost
      # postavi ju kao minimalnu
      min_distance = sum_distance
  for pattern, distance in distances.items(): # za svaku perm iz rijecnika
    # ako je zabiljezena vrijednost minimalna
    if distance == min_distance:
      # dodaj je u ispis
      output.append(pattern)
  return ' '.join(output) # rjesenja moze biti vise - vrati bilo koji

dnas = ['AAATTGACGCAT','GACGACCACGTT','CGTCAGCGCCTG','GCTGAGCACCGG','AGTACGGGACAG']
findMedianString(3, dnas)

'ACG GAC'

In [15]:
with open("rosalind_ba2b.txt", "r") as f:
  k = int(f.readline().strip())
  dna = [line.strip() for line in f]
findMedianString(k, dna)

'AATGTG'

## [BA2C](https://rosalind.info/problems/ba2c/) - Find a Profile-most Probable k-mer in a String

In [21]:
def probability(kmer, profile):
  lex = {'A': 0, 'C': 1, 'G': 2, 'T': 3} # red u matrici
  prob = 1
  for i in range(len(kmer)): # za svaku nukleotidu u kmeru
    row = lex.get(kmer[i]) # pronadi red u matrici za slovo
    prob *= profile[row][i] # pronadi poziciju u matrici profila
  return prob

In [18]:
def mostProbableKmer(string, k , profile):
  probs = {} # vjerojatnosti
  most_probable = [] # najvjerojatniji
  for i in range(len(string) - k + 1): # iteriranje po stringu
    pattern = string[i: i+k] # uzmi uzorak
    # dodaj u rjecnik vjerojatnost uzorka
    probs[pattern] = probability(pattern, profile)
  for kmer, prob in probs.items(): # za svaki uzorak
    if prob == max(probs.values()): # ako je vjerojatnost jednaka maksimalnoj
      most_probable.append(kmer) # dodaj kmer u listu
  # vrati najvjerojatnije
  return most_probable[0] if len(most_probable) == 1 else ' '.join(most_probable)

string = 'ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT'
k = 5
profile =[
[0.2, 0.2, 0.3, 0.2, 0.3],
[0.4, 0.3, 0.1, 0.5, 0.1],
[0.3, 0.3, 0.5, 0.2, 0.4],
[0.1, 0.2, 0.1, 0.1, 0.2]]
mostProbableKmer(string, k, profile)

'CCGAG'

In [20]:
with open("rosalind_ba2c.txt", "r") as f:
  dna = f.readline().strip()
  k = int(f.readline().strip())
  matrix =[[float(l) for l in line.strip().split()] for line in f]
mostProbableKmer(dna, k, matrix)

'TTGAGTT'

## [BA2D](https://rosalind.info/problems/ba2d/) - Implement GreedyMotifSearch

In [24]:
lex = {'A': 0, 'C': 1, 'G': 2, 'T': 3} # red profil matrice

In [25]:
def countMatrix(motifs, laplace):
  # inicijalizacija matrice
  if laplace:
    matrix = np.ones((4, len(motifs[0])))
  else:
    matrix = np.zeros((4, len(motifs[0])))
  # za svaki motiv u motivima
  for motif in motifs:
    # za svaki znak u motivu
    for i in range(len(motif)):
      # dodaj u matricu +1 za odgovorajauci red za znak
      matrix[lex.get(motif[i])][i] += 1
  return matrix

In [26]:
def createProfile(motifs, t, laplace = False):
  counts = countMatrix(motifs, laplace) # napravi matricu prebrojavanja
  devider = t + 4 if laplace else t # broj elemenata
  matrix = np.zeros((4, len(motifs[0]))) # inicijalizacija matrice profila
  for i in range(4): # za svaki red
    for j in range(len(motifs[0])): # za svaki stupac
      matrix[i][j] = counts[i][j] / devider # podijeli
  return matrix

In [27]:
def mostProbableKmer(k, profile, dna):
  max_prob = 0
  kmer = dna[0: k]
  for i in range(len(dna)-k+1): # iteriranje po dna
    pattern = dna[i: i+k] # uzorak duljine k
    prob = 1
    for j in range(k): # za svaki znak uzmi vjerojatnost iz matrice profila
      prob *= profile[lex.get(pattern[j])][j]
    if prob > max_prob: # ako je vjerojatnost veca od maksimalne
      max_prob = prob # postavi vjerojatnost kao maksimalnu
      kmer = pattern # postavi uzorak kao najvjerojatniji kmer
  return kmer

In [28]:
def createConcensus(profile):
  bases = ['A', 'C', 'G', 'T'] # baze
  concensus = ''
  for col in range(len(profile[0])): # za svaki stupac u matrici profila
    max_row, max_prob = 0, 0
    for row in range(len(profile)): # za svaki red u matrici profila
      # ako je vjerojatnost iz matrice profila veca od maksimalne
      if profile[row][col] > max_prob:
        max_prob = profile[row][col] # postavi novu vjerojatnost kao maksimalnu
        max_row = row # zapamti u kojem je redu najveca vjerojatnost
    concensus += bases[max_row] # u koncenzus dodaj odgovarajuce slovo
  return concensus

In [29]:
def Score(motifs, t):
  profile = createProfile(motifs, t) # profil motiva
  concensus = createConcensus(profile)  # koncenzus
  sc = 0 # inicijalizacija score
  for motif in motifs: # za svaki motiv u motivima
    for i in range(len(motif)): # za svaki znak u motivu
      if motif[i] != concensus[i]: # ako je znak razlicit od znaka u koncenzusu
        sc+=1 # povecaj score
  return sc

In [30]:
def greedyMotifSearch(k, t, dnas, laplace = False):
  # najbolji motivi su prvih k znakova svakog dna
  bestMotifs = [dna[0:k] for dna in dnas]
  # kmeri su svi k-meri prvog dna
  kmers = [dnas[0][i: i+k] for i in range(len(dnas[0])-k+1)]
  for kmer in kmers: # za svaki kmer iz prvog dna
    motifs = [kmer] # dodaj kmer u motive
    for dna in dnas[1:]: # za svaki dna u skupu dna (osim prvog)
      # napravi matricu profila sa dosadasnjim motivima
      profile = createProfile(motifs, t, laplace)
      # nadodaj najvjerojatnije motive po matrici profila
      motifs.append(mostProbableKmer(k, profile, dna))
    # ako je score dobivenih motiva u ovom koraku bolji od score dosadasnjih
    if Score(motifs, t) < Score(bestMotifs, t):
      # postavi dobivene motive u ovom koraku kao najbolji izbor
      bestMotifs = motifs
  return '\n'.join(bestMotifs) # vrati najvjerojatnije motive

In [31]:
greedyMotifSearch(3, 5,['GGCGTTCAGGCA',
                        'AAGAATCAGTCA',
                        'CAAGGAGTTCGC',
                        'CACGTCAATCAC',
                        'CAATAATATTCG'])

'CAG\nCAG\nCAA\nCAA\nCAA'

In [34]:
with open("rosalind_ba2d_2.txt", "r") as f:
  k, t = map(int, f.readline().strip().split())
  dna = [line.strip() for line in f]

greedyMotifSearch(k, t, dna)

'TACTTACGCAAT\nTCATTTTTCCGA\nGTTTTACGTTTC\nGTTTTCCGTGCG\nTCGTATGTTGAA\nGGCTGAAATGTG\nGCATTCAGCGAC\nGGTTGACGCAAG\nGAGTTCAGCGAT\nGGGTTCATCGAC\nGGCTTCAGCGAC\nGGATTCAGCGAT\nGATTTCATCGAC\nGAATTCAACGAA\nGCATTCAGCGAA\nGTCTTCATCGAA\nGGCTTCATCGAC\nGGTTTCATCGAA\nGATTTCAACGAA\nGAATTCAGCGAG\nGCGTTCAACGAG\nGACTTCAGCCGA\nGAATTCAACGAT\nGGGTTCAACGAC\nGACTTCAGCGAA'

## [BA2E](https://rosalind.info/problems/ba2e/) - Implement GreedyMotifSearch with Pseudocounts

In [35]:
#BA2D
dna = [
'GGCGTTCAGGCA',
'AAGAATCAGTCA',
'CAAGGAGTTCGC',
'CACGTCAATCAC',
'CAATAATATTCG']
k = 3
t = 5
greedyMotifSearch(k, t, dna, True)

'TTC\nATC\nTTC\nATC\nTTC'

In [39]:
with open("rosalind_ba2e.txt", "r") as f:
  k, t = map(int, f.readline().strip().split())
  dna = [line.strip() for line in f]
greedyMotifSearch(k, t, dna, True)

'ACCGAATATCAC\nAACCAATAACAA\nAACTAATATCAC\nAGCGAATAACAT\nAACAAATAACAG\nATCAAATAGCAC\nAGCAAATAGCAT\nAACTAATAGCAC\nAACTAATAGCAA\nAACGAATAGCAC\nAACAAATAGCAA\nAACGAATAGCAC\nAACTAATACCAC\nATCAAATACCAA\nAACCAATAACAG\nAACGAATAGCAT\nACCGAATATCAC\nACCTAATAACAA\nAGCAAATAGCAC\nAACGAATAGCAT\nAGCGAATAGCAG\nAACTAATACCAG\nATCTAATATCAG\nATCCAATATCAG\nATCAAATACCAA'

## [BA2F](https://rosalind.info/problems/ba2f/) - Implement RandomizedMotifSearch

In [42]:
def getRandom(n): # slucajni broj od 0 do n
  return randint(0, n)

In [43]:
def getRandomMotifs(k,dnas):
  kmers = []
  for dna in dnas: # za svaki dna u skupu dna uzoraka
    i = getRandom(len(dna)-k) # i je slucajan broj
    kmers.append(dna[i:i+k]) # dohvati kmer od slucajnog broja duljine k
  return kmers

In [44]:
def getMotifs(profile,dnas, k):
  motifs = []
  for dna in dnas: # za svaki dna u skupu dna uzoraka
    # BA2D
    motif = mostProbableKmer(k, profile, dna) # dohvati najvjerojatnije motive
    motifs.append(motif)
  return motifs

In [45]:
def randomizedMotifSearch_atom(k, t, dnas):
  bestMotifs = getRandomMotifs(k, dnas) # najbolji motivi su slucajni motivi
  motifs = bestMotifs
  while True:
    profile = createProfile(motifs, t, True) # kreiraj matricu profila (BA2D)
    motifs = getMotifs(profile, dnas, k) # dohvati motive
    if Score(motifs, t) < Score(bestMotifs, t): # ako je score manji (BA2D)
      bestMotifs = motifs
    else:
      return bestMotifs

In [46]:
def randomizedMotifSearch(k, t, dnas, n_repeats = 1000):
  bestMotifs = randomizedMotifSearch_atom(k, t, dnas)
  for repeat in range(1, n_repeats):
    motifs = randomizedMotifSearch_atom(k, t, dnas)
    if Score(motifs, t) < Score(bestMotifs, t):
      bestMotifs = motifs
  return '\n'.join(bestMotifs)

In [47]:
k = 8
t = 5
dnas = [
'CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA',
'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']

randomizedMotifSearch(k, t, dnas)

'TCTCGGGG\nCCAAGGTG\nTACAGGCG\nTTCAGGTG\nTCCACGTG'

In [50]:
with open("rosalind_ba2f.txt", "r") as f:
  k, t = map(int, f.readline().strip().split())
  dna = [line.strip() for line in f]
randomizedMotifSearch(k, t, dna, 1000)

'TACGATGTCATAGGA\nTACCGTTACATATCG\nTACGTATACATCCTG\nTACGTACGGATATCG\nTACGTATACAGGGCG\nTTGATATACATATCG\nTACACCTACATATCG\nTACGTAACGATATCG\nTACGTCCCCATATCG\nTACGTATACCGGTCG\nAACGTATACATATGT\nATCGTATACATATCT\nTACGTATTGCTATCG\nTACGTCGGCATATCG\nGGAGTATACATATCG\nTACGTATACATACTA\nTAGTCATACATATCG\nTACGCTGACATATCG\nTACGACAACATATCG\nTACGTATATGCATCG'

## [BA2G](https://rosalind.info/problems/ba2g/) - Implement GibbsSampler?

In [51]:
# vjerojatnost kmera po profilu
def probability(kmer, profile):
  prob = 1.0
  for i in range(len(kmer)):
    prob *= profile[lex.get(kmer[i])][i]
  return prob

In [57]:
# kmer po slucajnoj vjerojatnosti
def weigtedKmer(probs, dna, k):
  r = random.random()
  counter = 0.0
  prob_sum = sum(probs) # zbroji vjerojatnosti kmera
  for i in range(len(probs)):
    counter += probs[i] # dodaj vjerojatnost
    if counter / prob_sum > r: # ako je vjerojatnost veca od r
      return dna[i: i + k]

In [54]:
def formRandomMotif(k, dna, profile):
  probs = []
  for i in range(len(dna)-k+1):
    pattern = dna[i: i+k]
    probs.append(probability(pattern, profile)) # vjerojatnost kmera
  kmer = weigtedKmer(probs, dna, k)
  return kmer

In [55]:
def gibbsSampler_atom(k, t,N, dnas):
  bestMotifs = randomizedMotifSearch_atom(k, t, dnas) #BA2F
  motifs = bestMotifs
  for j in range(1,N):
    i = getRandom(t-1) # izbacujemo jedan (nasumicni) dna
    profile = createProfile(motifs[:i]+motifs[i+1:], t-1, True)
    motifs[i] = formRandomMotif(k, dnas[i], profile) # motiv za izbaceni dna
    if Score(motifs, t) < Score(bestMotifs, t):
      bestMotifs = motifs
    else:
      return bestMotifs

In [60]:
def gibbsSampler(k, t, N, dnas, random_starts = 20):
  bestKmers = gibbsSampler_atom(k, t, N, dnas)
  bestScore = Score(bestKmers, t)
  for i in range(1, random_starts):
    motifs = gibbsSampler_atom(k, t, N, dnas)
    if Score(motifs,t ) < bestScore:
      bestScore = Score(motifs, t)
      bestMotifs = motifs
  return '\n'.join(bestMotifs)

In [66]:
ans = gibbsSampler(8, 5, 100,['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA',
 'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
 'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
 'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
 'AATCCACCAGCTCCACGTGCAATGTTGGCCTA'], 20*20)
print(ans)

TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG


In [69]:
with open("rosalind_ba2g.txt", "r") as f:
  k, t, N= map(int, f.readline().strip().split())
  dna = [line.strip() for line in f]
gibbsSampler(k, t, N, dna, 20)

'GCGGACCTGTCGGGC\nGCGGACCTGTCGGGC\nGTGCACGAGTATGCG\nGTTCGAATTTCAGCG\nATACAGCTGTCAGAG\nATGGGCAATTGACAT\nATGGGCCCGTCAGGT\nGCGCGCTCGCCTGGG\nCCTCATCCGTCAACG\nACTTACGCGACAGCC\nGTTGACTTGCCGGCG\nGTGAGCGCGACAGGG\nGTGCACTCGACAGAT\nACGGTCTTGACCGCG\nGTGGGCGGGTGAGAG\nGTCAACCTGTCGGAG\nGTTCACCTCGGAGCC\nACGCAGATTTCAGCT\nGTTCGCAAGGCAGGG\nATACACCTGGCGGCA'

## [BA2H](https://rosalind.info/problems/ba2h/) - Implement DistanceBetweenPatternAndStrings

In [70]:
def hammingDistance(p, q):
  if len(p) != len(q):
    return -1
  return sum([1 for j in range(len(p))
                    if p[j] != q[j]])

In [71]:
def distanceBetweenPatternAndStrings(pattern, dna):
  distance = 0 # postavi udaljenost na nula
  k = len(pattern) # postavi k na duljinu uzorka
  for string in dna: # za svaki string u dna
    hamming_dist = float('inf') # postavi hamming udaljenost na beskonacno
    for i in range(len(string)-k+1): # iteriraj po stringu
      # izracunaj hamming udaljenost izmedu pocetnog uzorka i substringa
      new_dist = hammingDistance(pattern, string[i:i+k])
      if hamming_dist > new_dist: # ako je nova udaljenost manja od prosle
        # postavi novu hamming udaljenost na najnoviju
        hamming_dist = new_dist
    # povecaj ukupnu udaljenost za minimalnu udaljenost za svaki string iz dna
    distance = distance + hamming_dist
  return distance

distanceBetweenPatternAndStrings('AAA', ['TTACCTTAAC', 'GATATCTGTC', 'ACGGCGTTCG', 'CCCTAAAGAG', 'CGTCAGAGGT'])

5

In [72]:
with open("rosalind_ba2h.txt", "r") as f:
  pattern = f.readline().strip()
  strings = f.readline().strip().split(" ")
distanceBetweenPatternAndStrings(pattern, strings)

41